In [ ]:
import os
from giverny import turbulence_data_processing as turb_processing

# user-defined parameters for processing data.
# size of the model cube that data will be retrieved for.
cube_num = 8192
# number of dimensions that model data exists in.  default is 3 (i.e. X, Y, and Z dimensions).
cube_dimensions = 3
# turbulence dataset name, e.g. "isotropic8192" or "isotropic1024fine".
cube_title = 'isotropic8192'
# folder name to write the hdf5 output files to.
output_folder_name = 'turbulence_output'
# dask cluster ip address to use for parallel processing. to use an automated local cluster, simply 
# leave dask_cluster_ip as an empty string (''). to use a premade cluster, simply specify the scheduler ip address 
# of your premade cluster (e.g. '127.0.0.1:40770'). 
dask_cluster_ip = ''

# user specified box rather for which data values will be retrieved for each point inside the box.
# the user should specify the 1-based index range. this code will convert to 0-based index ranges for python.
x_range = [1, 512]
y_range = [1, 512]
z_range = [1, 512]

# variable of interest, currently set to velocity.
var = 'vel'
# time point. the user should specify the 1-based timepoint. this code will convert to a 0-based timepoint for python.
timepoint = 1

# folder path for this file. this will be the parent directory for where output files are stored. this parameter should
# not be changed by the user.
dir_path = os.path.dirname(os.path.realpath('__file__')) + '/'

# process the data.
output_data = turb_processing.turbulence_cube_processing(cube_num, cube_dimensions, cube_title, dir_path, output_folder_name, \
                                                         x_range, y_range, z_range, var, timepoint, dask_cluster_ip)

In [ ]:
# retrieve the data value for a datapoint (X, Y, Z). the points use a 1-based index.
X = 4
Y = 5
Z = 8

data_value = turb_processing.retrieve_data_for_point(X, Y, Z, output_data, x_range, y_range, z_range)

print(f'data value ("{var}") for datapoint ({X}, {Y}, {Z}):\n{data_value}\n' + '-' * 5)